In [1]:
import spotipy
import spotipy.util as util
import json
import pandas as pd
import numpy as np

with open('config.json') as json_file:
    data = json.load(json_file)

spotify_username = data['spotify_username']
lastfm_username = data['lastfm_username']
lastfm_password = data['lastfm_password']
client_id = data['client_id']
client_secret = data['client_secret']
API_KEY = data['API_KEY']
API_SECRET = data['API_SECRET']

scope = 'user-library-read'

token = util.prompt_for_user_token(spotify_username,scope,client_id,client_secret,redirect_uri='https://google.com/')
if token:
    sp = spotipy.Spotify(auth=token)
    print("login successful.")
else:
    print ("Can't get token for", spotify_username)

songs = []
song_ids = []

full_genres = [['soundtrack'],
                ['jazz'],
                ['classical'],
               ['metal'],
               ['indietronica','wave','synth'],
               ['downtempo','trip hop'],
               ['edm','electronica','idm','dubstep','techno'],
               ['house'],
               ['r&b','rnb','soul'],
               ['rock'],
               ['hip hop','rap','trap','hiphop'],
               ['pop']]

login successful.


In [ ]:
def saveSongList(filename):
    with open(filename, "w", encoding="utf8") as outfile:
        global songs
        json.dump(songs,outfile)

def loadSongList(filename):
    with open(filename, 'r') as myfile:
        global songs
        data=myfile.read().replace('\n', '')
        song_data  = json.loads(data)
        songs = song_data
        extractSongIds()
        
def extractSongIds():
        global song_ids
        song_ids = [row[1] for row in songs]
        
def saveDataFrame(filename):
    df.to_pickle(filename)

def loadDataFrame(filename):
    df = pd.read_pickle(filename)

In [ ]:
playlists = [
['c2thorn','2eANB3B0lHY5F8MgbH1qlT'],
['S4INT7', '3altbbY28EdjcPI6hAppwl'],
['1251339404', '6tACOOyeaY5JeQ9dVDTHkT'],
['thesoundsofspotify', '2RmQ1WAONeEih0FWEK7CW5'],
['thesoundsofspotify', '5EyFMotmvSfDAZ4hSdKrbx'],
['thesoundsofspotify', '3HYK6ri0GkvRcM6GkKh0hJ'],
['thesoundsofspotify', '3pBfUFu8MkyiCYyZe849Ks'],
['thesoundsofspotify', '0yqVOsxA2U4P260ad60QuU'],
['thesoundsofspotify', '5zxj55xrB53As97uvt5x8x'],
['thesoundsofspotify', '3pDxuMpz94eDs7WFqudTbZ'],
['thesoundsofspotify', '6AzCASXpbvX5o3F8yaj1y0'],
['thesoundsofspotify', '1rLnwJimWCmjp3f0mEbnkY'],
['thesoundsofspotify', '7dowgSWOmvdpwNkGFMUs6e'],
['thesoundsofspotify', '6MXkE0uYF4XwU4VTtyrpfP'],
['thesoundsofspotify', '6gS3HhOiI17QNojjPuPzqc']
]

for j,playlist in enumerate(playlists): 
    playlist_data = sp.user_playlist(playlist[0],playlist[1])
    playlist_name = playlist_data['name']
    playlist_size = playlist_data['tracks']['total']

    print('*********************',playlist_name,'|',j, 'of ', len(playlists), ' playlists','*********************')
    
    for i in range(0,playlist_size,100):
        print(i,'out of',playlist_size)
        try:
            playlist_items = sp.user_playlist_tracks(playlist[0], playlist[1],offset=i)['items']
        except:
            print('Connection error')
            i = i-100
            continue
        for item in playlist_items:
            song = []
            if item is None:
                continue
            if item['track'] is None:
                continue
            song.append(item['track']['name'])
            song.append(item['track']['uri'])
            song.append(item['track']['artists'][0]['name'])
            song.append(item['track']['album']['name'])
            artist_id = item['track']['artists'][0]['uri']
            try:
                artist_info = sp.artist(artist_id)
            except:
                continue
            song.append([len(full_genres),'other'])
            matchFound = False
            for j,genres in enumerate(full_genres):
                for artist_genre in artist_info['genres']:
                    for genre in genres:
                        if artist_genre in genre:
                            song[-1] = [j,genres[0]]
                            matchFound = True
                            break;
                    if matchFound:
                        break;
                if matchFound:
                    break;
            if matchFound:
                songs.append(song)
print(len(songs))

ids = [row[1] for row in songs]

********************* All Songs | 0 of  15  playlists *********************
0 out of 4027
100 out of 4027
200 out of 4027
300 out of 4027
400 out of 4027
500 out of 4027
600 out of 4027


From cffi callback <function _verify_callback at 0x0000027E2B284488>:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\OpenSSL\SSL.py", line 310, in wrapper
    @wraps(callback)
KeyboardInterrupt


Connection error
700 out of 4027


In [ ]:
import pylast as pylast
import re

username = "c2thorn"
password_hash = pylast.md5(lastfm_password)

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
                               username=lastfm_username, password_hash=password_hash)


for i in range(len(songs)):
    if i % 500 == 0:
        print('*********************',i, 'of ', len(songs), ' songs','*********************')
    song = songs[i]
    track = network.get_track(song[2],song[0])
    try:
        topItems = track.get_top_tags(limit=3)
    except:
        topItems = []
    matchFound = False
    for item in topItems:
        tag = re.findall(r"'(.*?)'", str(item))[0]
        for j,genres in enumerate(full_genres):
            for genre in genres:
                if genre.lower().replace('-',' ') in tag:
#                     print(tag,'|',genres[0],'\tBEFORE:',song[-1][1],'\tSONG:',song[0],song[2])
                    song[-1] = [j,genres[0]]
                    matchFound = True
                    break;
            if matchFound:
                break;
        if matchFound:
            break;

# saveData('song_list_with_lastfm_tags.txt')

ids = [row[1] for row in songs]

In [ ]:
audio_features= np.zeros((0,10))

raw_audio_features = []
print(len(songs))
print(len(ids))
for k in range(0,len(songs),100):
    raw_audio_features.extend(sp.audio_features(ids[k:k+100]))
    

for i,feature in enumerate(raw_audio_features):
    song_data = []
    song_data.append(feature['danceability'])
    song_data.append(feature['energy'])
    song_data.append(feature['loudness'])
    song_data.append(feature['speechiness'])
    song_data.append(feature['acousticness'])
    song_data.append(feature['instrumentalness'])
    song_data.append(feature['liveness'])
    song_data.append(feature['valence'])
    song_data.append(feature['tempo'])
    song_data.append(songs[i][-1][0])
    audio_features = np.append(audio_features,[song_data],axis=0)

column_names= ['danceability','energy','loudness','speechiness',
          'acousticness','instrumentalness','liveness','valence','tempo','genre']    

df = pd.DataFrame(data=audio_features[0:,0:],
                 columns=column_names)
print(df.size)
df[:10]

In [ ]:
saveSongList('song_list_final.txt')
saveDataFrame('dataframe.pkl')